In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display
from scipy.stats import mannwhitneyu, wilcoxon

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
from lib.sych.plot_helper import imshow_dataset_by_mouse, imshow_dataset_by_session
from lib.sych.metric_helper import metric_by_session, metric_by_selector

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [4]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
#params['root_path_data'] = gui_fpath('h5path', './')

In [5]:
dataDB = DataFCDatabase(params)

Searching for data files


In [6]:
ds = DataStorage('sych_result_bulk_metrics.h5')

In [7]:
mc = MetricCalculator(serial=True, verbose=False)

# 1. Bulk Metrics vs Time

* Avg over mice
* All-region-all-types
* Expert vs Naive
* Go/NOGO/MISS/FA

In [14]:
none2all = lambda x: x if x is not None else 'All'

def metric_mouse_bulk(dataDB, mc, ds, metricName, dimOrdTrg, nameSuffix,
                              metricSettings=None, sweepSettings=None,
                              trialTypeNames=None, perfNames=None, cropTime=None):
    for mousename in dataDB.mice:
        for datatype in dataDB.get_data_types(mousename):
            perfNames = ['naive', 'expert'] if perfNames is None else perfNames
            
            for performance in [None] + perfNames:
                trialTypeNames = dataDB.get_trial_type_names(mousename) if trialTypeNames is None else trialTypeNames
                
                for trialType in [None] + trialTypeNames:
                    dataName = '_'.join([metricName, nameSuffix, datatype, none2all(performance), none2all(trialType)])
                    zscoreDim = 'rs' if datatype == 'raw' else None
                    
                    print(mousename, dataName)
                    
                    metric_by_selector(dataDB, mc, ds, {'mousename' : mousename}, metricName, dimOrdTrg,
                                       dataName=dataName, datatype=datatype, trialType=trialType, cropTime=cropTime,
                                       performance=performance,
                                       zscoreDim=zscoreDim,
                                       metricSettings=metricSettings,
                                       sweepSettings=sweepSettings)
                    
                    
def metric_mouse_bulk_vs_session(dataDB, mc, ds, metricName, dimOrdTrg,
                                 metricSettings=None, sweepSettings=None, trialTypeNames=None):
    for mousename in dataDB.mice:
        for datatype in dataDB.get_data_types(mousename):
            for performance in [None, 'naive', 'expert']:
                if trialTypeNames is None:
                    trialTypeNamesThis = dataDB.get_trial_type_names(mousename)
                else:
                    trialTypeNamesThis = trialTypeNames
                
                for trialType in [None] + trialTypeNamesThis:
                    dataName = '_'.join([metricName, 'session', datatype, none2all(performance), none2all(trialType)])
                    zscoreDim = 'rs' if datatype == 'raw' else None
                    
                    print(mousename, dataName)
                    
                    metric_by_session(dataDB, mc, ds, mousename, metricName, dimOrdTrg,
                                      dataName=dataName, datatype=datatype, trialType=trialType,
                                      performance=performance,
                                      zscoreDim=zscoreDim,
                                      metricSettings=metricSettings,
                                      sweepSettings=sweepSettings)
                    
                    
def plot_metric_bulk(dataDB, mc, ds, metricName, nameSuffix, prepFunc=None, ylim=None, yscale=None):
    dummyMouseName = 'mvg_4'
    
    dfAll = ds.list_dsets_pd()
    for datatype in dataDB.get_data_types(dummyMouseName):    
        for performance in [None, 'naive', 'expert']:
            for trialType in [None] + dataDB.get_trial_type_names(dummyMouseName):
                dataName = '_'.join([metricName, nameSuffix, datatype, none2all(performance), none2all(trialType)])
                dfThis = dfAll[dfAll['name'] == dataName]
                dfThis = dfThis.sort_values(by=['mousename'])
                
                print(dataName)
                
                if len(dfThis) == 0:
                    print('--Nothing found, skipping')
                else:                
                    plt.figure()
                    for idx, row in dfThis.iterrows():
                        dataThis = ds.get_data(row['dset'])
                        if prepFunc is not None:
                            dataThis = prepFunc(dataThis)

    #                     if datatype == 'raw':
    #                         nTrialThis = dataDB.get_ntrial_bytype({'mousename' : row['mousename']}, trialType=trialType, performance=performance)
    #                         dataThis *= np.sqrt(48*nTrialThis)
    #                         print('--', row['mousename'], nTrialThis)
    
                        if nameSuffix == 'time':
                            plt.plot(np.arange(0, 8, 1/20), dataThis, label=row['mousename'])
                        else:
                            plt.plot(dataThis, label=row['mousename'])

                    if yscale is not None:
                        plt.yscale(yscale)
                            
                    plt.legend()
                    plt.ylim(ylim)
                    plt.savefig('pics/'+ dataName + '.pdf')
                    plt.close()
                
                
def plot_metric_bulk_vs_session(dataDB, mc, ds, metricName, trialTypeNames=None, ylim=None):
    dummyMouseName = 'mvg_4'
    
    dfAll = ds.list_dsets_pd()
    for datatype in dataDB.get_data_types(dummyMouseName):    
        for performance in [None, 'naive', 'expert']:
            if trialTypeNames is None:
                trialTypeNamesThis = dataDB.get_trial_type_names(dummyMouseName)
            else:
                trialTypeNamesThis = trialTypeNames
            
            for trialType in [None] + trialTypeNamesThis:
                dataName = '_'.join([metricName, 'session', datatype, none2all(performance), none2all(trialType)])
                dfThis = dfAll[dfAll['name'] == dataName]
                dfThis = dfThis.sort_values(by=['mousename'])
                
                print(dataName, len(dfThis))
                
                plt.figure()
                for idx, row in dfThis.iterrows():
                    dataThis = ds.get_data(row['dset'])
                    dataThis = np.mean(dataThis, axis=1)
                    plt.plot(dataThis, label=row['mousename'])
                    
                plt.legend()
                plt.ylim(ylim)
                plt.savefig('pics/'+ dataName + '.pdf')
                plt.close()

# Mean

In [9]:
metric_mouse_bulk(dataDB, mc, ds, 'mean', 's', 'time')

mvg_8 mean_time_bn_session_All_All
mvg_8 mean_time_bn_session_All_iGO
mvg_8 mean_time_bn_session_All_iNOGO
mvg_8 mean_time_bn_session_All_iFA
mvg_8 mean_time_bn_session_All_iMISS
mvg_8 mean_time_bn_session_naive_All
mvg_8 mean_time_bn_session_naive_iGO
mvg_8 mean_time_bn_session_naive_iNOGO
mvg_8 mean_time_bn_session_naive_iFA
mvg_8 mean_time_bn_session_naive_iMISS
mvg_8 mean_time_bn_session_expert_All
mvg_8 mean_time_bn_session_expert_iGO
mvg_8 mean_time_bn_session_expert_iNOGO
mvg_8 mean_time_bn_session_expert_iFA
mvg_8 mean_time_bn_session_expert_iMISS
mvg_8 mean_time_bn_trial_All_All
mvg_8 mean_time_bn_trial_All_iGO
mvg_8 mean_time_bn_trial_All_iNOGO
mvg_8 mean_time_bn_trial_All_iFA
mvg_8 mean_time_bn_trial_All_iMISS
mvg_8 mean_time_bn_trial_naive_All
mvg_8 mean_time_bn_trial_naive_iGO
mvg_8 mean_time_bn_trial_naive_iNOGO
mvg_8 mean_time_bn_trial_naive_iFA
mvg_8 mean_time_bn_trial_naive_iMISS
mvg_8 mean_time_bn_trial_expert_All
mvg_8 mean_time_bn_trial_expert_iGO
mvg_8 mean_time_bn

In [14]:
plot_metric_bulk(dataDB, mc, ds, 'mean', 'time')

mean_time_bn_session_All_All
mean_time_bn_session_All_iGO
mean_time_bn_session_All_iNOGO
mean_time_bn_session_All_iFA
mean_time_bn_session_All_iMISS
mean_time_bn_session_naive_All
mean_time_bn_session_naive_iGO
mean_time_bn_session_naive_iNOGO
mean_time_bn_session_naive_iFA
mean_time_bn_session_naive_iMISS
mean_time_bn_session_expert_All
mean_time_bn_session_expert_iGO
mean_time_bn_session_expert_iNOGO
mean_time_bn_session_expert_iFA
mean_time_bn_session_expert_iMISS
mean_time_bn_trial_All_All
mean_time_bn_trial_All_iGO
mean_time_bn_trial_All_iNOGO
mean_time_bn_trial_All_iFA
mean_time_bn_trial_All_iMISS
mean_time_bn_trial_naive_All
mean_time_bn_trial_naive_iGO
mean_time_bn_trial_naive_iNOGO
mean_time_bn_trial_naive_iFA
mean_time_bn_trial_naive_iMISS
mean_time_bn_trial_expert_All
mean_time_bn_trial_expert_iGO
mean_time_bn_trial_expert_iNOGO
mean_time_bn_trial_expert_iFA
mean_time_bn_trial_expert_iMISS
mean_time_raw_All_All
mean_time_raw_All_iGO
mean_time_raw_All_iNOGO
mean_time_raw_All_

# Variance

**TODO**
* Plot trial variance relative to temporal variance
* Plot mean with variance together

## Pros/Cons of Baseline Normalization
* DFF-Trial
    - Pos: Removes dynamic baseline changing on the order of trials.
    - Pos: Under assumption of signal-free pre-trial interval, baseline removal enhances relative change in significant activity during trial.
    - Neg: In presence of correlation between pre-trial interval and trial signals, this procedure destroys information during trial.

* DFF-Session vs ZScore-Session
    - Both linear transforms
    - Mean is more meaningful for DFF if pre-trial interval is at least somewhat stable
    - Va

In [17]:
metric_mouse_bulk(dataDB, mc, ds, 'varmean', 's', 'time')

mvg_9 varmean_time_bn_session_All_All
mvg_9 varmean_time_bn_session_All_iGO
mvg_9 varmean_time_bn_session_All_iNOGO
mvg_9 varmean_time_bn_session_All_iFA
mvg_9 varmean_time_bn_session_All_iMISS
mvg_9 varmean_time_bn_session_naive_All
mvg_9 varmean_time_bn_session_naive_iGO
mvg_9 varmean_time_bn_session_naive_iNOGO
mvg_9 varmean_time_bn_session_naive_iFA
mvg_9 varmean_time_bn_session_naive_iMISS
mvg_9 varmean_time_bn_session_expert_All
mvg_9 varmean_time_bn_session_expert_iGO
mvg_9 varmean_time_bn_session_expert_iNOGO
mvg_9 varmean_time_bn_session_expert_iFA
mvg_9 varmean_time_bn_session_expert_iMISS
mvg_9 varmean_time_bn_trial_All_All
mvg_9 varmean_time_bn_trial_All_iGO
mvg_9 varmean_time_bn_trial_All_iNOGO
mvg_9 varmean_time_bn_trial_All_iFA
mvg_9 varmean_time_bn_trial_All_iMISS
mvg_9 varmean_time_bn_trial_naive_All
mvg_9 varmean_time_bn_trial_naive_iGO
mvg_9 varmean_time_bn_trial_naive_iNOGO
mvg_9 varmean_time_bn_trial_naive_iFA
mvg_9 varmean_time_bn_trial_naive_iMISS
mvg_9 varmean_t

In [18]:
# rms2D = lambda x: np.linalg.norm(x, axis=0) / np.sqrt(len(x))
# plot_metric_bulk(dataDB, mc, ds, 'std', 'time', prepFunc=rms2D, ylim=[0,None])
plot_metric_bulk(dataDB, mc, ds, 'varmean', 'time', ylim=[0,None])

varmean_time_bn_session_All_All
varmean_time_bn_session_All_iGO
varmean_time_bn_session_All_iNOGO
varmean_time_bn_session_All_iFA
varmean_time_bn_session_All_iMISS
varmean_time_bn_session_naive_All
varmean_time_bn_session_naive_iGO
varmean_time_bn_session_naive_iNOGO
varmean_time_bn_session_naive_iFA
varmean_time_bn_session_naive_iMISS
varmean_time_bn_session_expert_All
varmean_time_bn_session_expert_iGO
varmean_time_bn_session_expert_iNOGO
varmean_time_bn_session_expert_iFA
varmean_time_bn_session_expert_iMISS
varmean_time_bn_trial_All_All
varmean_time_bn_trial_All_iGO
varmean_time_bn_trial_All_iNOGO
varmean_time_bn_trial_All_iFA
varmean_time_bn_trial_All_iMISS
varmean_time_bn_trial_naive_All
varmean_time_bn_trial_naive_iGO
varmean_time_bn_trial_naive_iNOGO
varmean_time_bn_trial_naive_iFA
varmean_time_bn_trial_naive_iMISS
varmean_time_bn_trial_expert_All
varmean_time_bn_trial_expert_iGO
varmean_time_bn_trial_expert_iNOGO
varmean_time_bn_trial_expert_iFA
varmean_time_bn_trial_expert_iM

### Test variance across channels for interesting interval

* Average signal over texture presentation interval
* Compute variance over trials for each channel
* Compare channels

In [9]:
metric_mouse_bulk(dataDB, mc, ds, 'std', 'p', 'channel', cropTime=(6, 6.5), trialTypeNames=[], perfNames=[])

mvg_7 std_channel_bn_session_All_All
mvg_7 std_channel_bn_trial_All_All
mvg_7 std_channel_raw_All_All
mvg_9 std_channel_bn_session_All_All
mvg_9 std_channel_bn_trial_All_All
mvg_9 std_channel_raw_All_All
mvg_8 std_channel_bn_session_All_All
mvg_8 std_channel_bn_trial_All_All
mvg_8 std_channel_raw_All_All
mvg_4 std_channel_bn_session_All_All
mvg_4 std_channel_bn_trial_All_All
mvg_4 std_channel_raw_All_All


In [15]:
# rms2D = lambda x: np.linalg.norm(x, axis=0) / np.sqrt(len(x))
plot_metric_bulk(dataDB, mc, ds, 'std', 'channel', yscale='log', ylim=[0.005,2])

std_channel_bn_session_All_All
std_channel_bn_session_All_iGO
--Nothing found, skipping
std_channel_bn_session_All_iNOGO
--Nothing found, skipping
std_channel_bn_session_All_iFA
--Nothing found, skipping
std_channel_bn_session_All_iMISS
--Nothing found, skipping
std_channel_bn_session_naive_All
--Nothing found, skipping
std_channel_bn_session_naive_iGO
--Nothing found, skipping
std_channel_bn_session_naive_iNOGO
--Nothing found, skipping
std_channel_bn_session_naive_iFA
--Nothing found, skipping
std_channel_bn_session_naive_iMISS
--Nothing found, skipping
std_channel_bn_session_expert_All
--Nothing found, skipping
std_channel_bn_session_expert_iGO
--Nothing found, skipping
std_channel_bn_session_expert_iNOGO
--Nothing found, skipping
std_channel_bn_session_expert_iFA
--Nothing found, skipping
std_channel_bn_session_expert_iMISS
--Nothing found, skipping
std_channel_bn_trial_All_All
std_channel_bn_trial_All_iGO
--Nothing found, skipping
std_channel_bn_trial_All_iNOGO
--Nothing found, sk

# Effective Rank

### ByTime

In [7]:
metric_mouse_bulk(dataDB, mc, ds, "rank_effective", 's', 'time')

mvg_4 rank_effective_time_bn_session_All_All
mvg_4 rank_effective_time_bn_session_All_iGO
mvg_4 rank_effective_time_bn_session_All_iNOGO
mvg_4 rank_effective_time_bn_session_All_iFA
mvg_4 rank_effective_time_bn_session_All_iMISS
mvg_4 rank_effective_time_bn_session_naive_All
mvg_4 rank_effective_time_bn_session_naive_iGO
mvg_4 rank_effective_time_bn_session_naive_iNOGO
mvg_4 rank_effective_time_bn_session_naive_iFA
mvg_4 rank_effective_time_bn_session_naive_iMISS
mvg_4 rank_effective_time_bn_session_expert_All
mvg_4 rank_effective_time_bn_session_expert_iGO
mvg_4 rank_effective_time_bn_session_expert_iNOGO
mvg_4 rank_effective_time_bn_session_expert_iFA
mvg_4 rank_effective_time_bn_session_expert_iMISS
mvg_4 rank_effective_time_bn_trial_All_All
mvg_4 rank_effective_time_bn_trial_All_iGO
mvg_4 rank_effective_time_bn_trial_All_iNOGO
mvg_4 rank_effective_time_bn_trial_All_iFA
mvg_4 rank_effective_time_bn_trial_All_iMISS
mvg_4 rank_effective_time_bn_trial_naive_All
mvg_4 rank_effective_tim

In [49]:
plot_metric_bulk(dataDB, mc, ds, 'rank_effective', 'time', ylim=[1,48])

rank_effective_time_bn_session_All_All
rank_effective_time_bn_session_All_iGO
rank_effective_time_bn_session_All_iNOGO
rank_effective_time_bn_session_All_iFA
rank_effective_time_bn_session_All_iMISS
rank_effective_time_bn_session_naive_All
rank_effective_time_bn_session_naive_iGO
rank_effective_time_bn_session_naive_iNOGO
rank_effective_time_bn_session_naive_iFA
rank_effective_time_bn_session_naive_iMISS
rank_effective_time_bn_session_expert_All
rank_effective_time_bn_session_expert_iGO
rank_effective_time_bn_session_expert_iNOGO
rank_effective_time_bn_session_expert_iFA
rank_effective_time_bn_session_expert_iMISS
rank_effective_time_bn_trial_All_All
rank_effective_time_bn_trial_All_iGO
rank_effective_time_bn_trial_All_iNOGO
rank_effective_time_bn_trial_All_iFA
rank_effective_time_bn_trial_All_iMISS
rank_effective_time_bn_trial_naive_All
rank_effective_time_bn_trial_naive_iGO
rank_effective_time_bn_trial_naive_iNOGO
rank_effective_time_bn_trial_naive_iFA
rank_effective_time_bn_trial_na

### BySession

In [37]:
metric_mouse_bulk_vs_session(dataDB, mc, ds, "rank_effective", trialTypeNames=[])

mvg_7 rank_effective_session_bn_session_All_All


IntProgress(value=0, description='mvg_7', max=18)

mvg_7 rank_effective_session_bn_session_naive_All


IntProgress(value=0, description='mvg_7', max=15)

mvg_7 rank_effective_session_bn_session_expert_All


...

IntProgress(value=0, description='mvg_9', max=21)

mvg_9 rank_effective_session_raw_naive_All


IntProgress(value=0, description='mvg_9', max=19)

mvg_9 rank_effective_session_raw_expert_All


IntProgress(value=0, description='mvg_9', max=2)

In [50]:
plot_metric_bulk_vs_session(dataDB, mc, ds, 'rank_effective', trialTypeNames=[], ylim=[1,48])

rank_effective_session_bn_session_All_All 4
rank_effective_session_bn_session_naive_All 4
rank_effective_session_bn_session_expert_All 4
rank_effective_session_bn_trial_All_All 4
rank_effective_session_bn_trial_naive_All 4
rank_effective_session_bn_trial_expert_All 4
rank_effective_session_raw_All_All 4
rank_effective_session_raw_naive_All 4
rank_effective_session_raw_expert_All 4


### Cleaning up

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ds.list_dsets_pd().sort_values(by='datetime'))

,datetime,dset,metric,mousename,name,shape,target_dim
0,2020-11-18 08:13:13,dset0,mean,mvg_8,mean_time_bn_session_All_All,"(160,)","(timesteps,)"
1,2020-11-18 08:13:13,dset1,mean,mvg_8,mean_time_bn_session_All_iGO,"(160,)","(timesteps,)"
112,2020-11-18 08:13:13,dset2,mean,mvg_8,mean_time_bn_session_All_iNOGO,"(160,)","(timesteps,)"
223,2020-11-18 08:13:13,dset3,mean,mvg_8,mean_time_bn_session_All_iFA,"(160,)","(timesteps,)"
334,2020-11-18 08:13:13,dset4,mean,mvg_8,mean_time_bn_session_All_iMISS,"(160,)","(timesteps,)"
35,2020-11-18 08:13:14,dset13,mean,mvg_8,mean_time_bn_session_expert_iFA,"(160,)","(timesteps,)"
543,2020-11-18 08:13:14,dset7,mean,mvg_8,mean_time_bn_session_naive_iNOGO,"(160,)","(timesteps,)"
46,2020-11-18 08:13:14,dset14,mean,mvg_8,mean_time_bn_session_expert_iMISS,"(160,)","(timesteps,)"
532,2020-11-18 08:13:14,dset6,mean,mvg_8,mean_time_bn_session_naive_iGO,"(160,)","(timesteps,)"
445,2020-11-18 08:13:14,dset5,mean,mvg_8,mean_time_bn_session_naive_All,"(160,)","(timesteps,)"


In [6]:
ds.delete_recent("2020-11-23 08:00:00")

Deleting 0 rows
